In [76]:
from itertools import combinations

from keras import layers
from tensorflow import keras
from xgboost import XGBClassifier
import numpy as np
import tensorflow as tf
import pandas as pd
from sklearn import preprocessing

In [82]:
x_test = pd.read_csv('x_test.csv')
x_train = pd.read_csv('x_train.csv')
y_train = pd.read_csv('y_train.csv')

In [87]:
x_tn = x_train.values

asign = lambda t: 0 if t<0 else 1
list(map(asign, x_tn[0,7:46:2]))

[0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0]

In [89]:
interval = 0  # Used for iteration
OBV_Value = []  # Sets the initial OBV_Value to zero
count = 0
x_tn = x_train.values
for i in range(x_train.shape[0]):
    OBV_Value.append(np.sum(x_tn[i,8:47:2]*list(map(asign, x_tn[0,7:46:2]))))#/x_train[i,45]
OBV_Value = pd.DataFrame(np.array(OBV_Value).reshape(len(OBV_Value),-1),columns=["OBV_Value"])
x_train = pd.concat([x_train,OBV_Value],axis=1)

In [72]:
def moving_average(x_train,window):
    row = x_train.iloc[:,7:46:2].rolling(window).mean()
    print(row.size)
    for i in range(1,21-window):
        x_train[f'MA{window}_{i}'] = row.iloc[:, 21-window-i]
# removing all the NULL values using
# dropna() method
    x_train.dropna(inplace=True)
    print(x_train)

In [73]:
moving_average(x_train.copy(),5)

8371900
            ID  DATE  STOCK  INDUSTRY  INDUSTRY_GROUP  SECTOR  SUB_INDUSTRY  \
4            4     0     14        36              12       5            92   
5            5     0     23        37              12       5            94   
6            6     0     45        52              18       7           121   
7            7     0     59        57              20       8           143   
9            9     0     65        56              20       8           139   
...        ...   ...    ...       ...             ...     ...           ...   
418590  418590   223   5703        32              10       4            77   
418591  418591   223   5705        35              12       5            91   
418592  418592   223   5709         2               1       1             5   
418593  418593   223   5710        33              10       4            83   
418594  418594   223   5713        26               7       4            60   

           RET_1  VOLUME_1     RET_2  ...  

In [67]:
x_train

,ID,DATE,STOCK,INDUSTRY,INDUSTRY_GROUP,SECTOR,SUB_INDUSTRY,RET_1,VOLUME_1,RET_2,...,RET_16,VOLUME_16,RET_17,VOLUME_17,RET_18,VOLUME_18,RET_19,VOLUME_19,RET_20,VOLUME_20
0,0,0,2,18,5,3,44,-0.015748,0.147931,-0.015504,...,0.059459,0.630899,0.003254,-0.379412,0.008752,-0.110597,-0.012959,0.174521,-0.002155,-0.000937
1,1,0,3,43,15,6,104,0.003984,NaN,-0.090580,...,0.015413,NaN,0.003774,NaN,-0.018518,NaN,-0.028777,NaN,-0.034722,NaN
2,2,0,4,57,20,8,142,0.000440,-0.096282,-0.058896,...,0.008964,-0.010336,-0.017612,-0.354333,-0.006562,-0.519391,-0.012101,-0.356157,-0.006867,-0.308868
3,3,0,8,1,1,1,2,0.031298,-0.429540,0.007756,...,-0.031769,0.012105,0.033824,-0.290178,-0.001468,-0.663834,-0.013520,-0.562126,-0.036745,-0.631458
4,4,0,14,36,12,5,92,0.027273,-0.847155,-0.039302,...,-0.038461,-0.277083,-0.012659,0.139086,0.004237,-0.017547,0.004256,0.579510,-0.040817,0.802806
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
418590,418590,223,5703,32,10,4,77,0.021843,-0.217823,-0.021703,...,0.003160,-0.161543,0.007785,-0.141487,0.008205,-0.268215,-0.058448,0.057577,0.031174,-0.163350
418591,418591,223,5705,35,12,5,91,-0.006920,-0.375251,0.000000,...,0.007169,-0.955492,-0.016221,-0.171172,0.007819,1.540184,-0.015396,-0.502337,-0.011073,0.632885
418592,418592,223,5709,2,1,1,5,0.021869,-0.978856,-0.005929,...,0.001110,-0.476550,0.029714,-0.385972,-0.069148,1.780169,-0.082927,1.581453,0.098607,6.956960
418593,418593,223,5710,33,10,4,83,0.012248,-0.627169,0.010925,...,0.007450,-0.210079,0.023729,0.813948,0.026087,-0.868576,-0.051155,0.371052,0.041238,0.045695


In [24]:
def scaler(data):
    data = data.fillna(data.mean())
    no_scale = data.iloc[:, :6]
    to_scale = data.iloc[:, 6:]
    temp = to_scale.columns

    # Scalling
    min_max_scaler = preprocessing.MinMaxScaler()
    to_scale = min_max_scaler.fit_transform(to_scale)
    to_scale = pd.DataFrame(to_scale)
    to_scale.columns = temp
    data = pd.concat([no_scale, to_scale], axis=1)

    return data

In [25]:
x_train = scaler(x_train)
x_test = scaler(x_test)

In [12]:
def clean_data(X):
    i_v = []
    i_r = []
    for i in range(20):
        i_v.append(r'VOLUME_{}'.format(i + 1))
        i_r.append(r'RET_{}'.format(i + 1))

    x = {}
    x[0] = X[i_v].T
    x[1] = X[i_r].T
    x[2] = X[['ID', 'DATE', 'STOCK', 'INDUSTRY', 'INDUSTRY_GROUP', 'SECTOR', 'SUB_INDUSTRY']].T

    for i in range(2):
        x[i] = (x[i].fillna(method="ffill") + x[i].fillna(method="bfill")) / 2
        x[i].iloc[:, -1].fillna(method="ffill", inplace=True)
        x[i].iloc[:, 0].fillna(method="bfill", inplace=True)
        x[i].fillna(0, inplace=True)
    x[2] = x[2].fillna(0)
    x_result = pd.concat([x[2], x[0], x[1]])
    return x_result.T


def weekly_means(data):
    meanx_train = data.copy()
    ret_cols = []
    for ret in meanx_train.columns:
        if "RET" in ret:
            ret_cols.append(ret)

    vol_cols = []
    for vol in meanx_train.columns:
        if "VOLUME" in vol:
            vol_cols.append(vol)

    list_1 = [0, 5, 10, 15]
    list_2 = [5, 10, 15, 20]

    for i in range(4):
        temp1 = ret_cols[list_1[i]:list_2[i]]
        temp2 = vol_cols[list_1[i]:list_2[i]]

        meanx_train['average R Week ' + str(i)] = meanx_train[temp1].mean(axis=1)
        meanx_train['average V Week ' + str(i)] = meanx_train[temp2].mean(axis=1)

    return pd.concat([meanx_train.iloc[:, :7], meanx_train.iloc[:, 47:]], axis=1)


def data_permute(x_train):
    x_train = clean_data(x_train)
    feature_to_gbs = ['INDUSTRY', 'SECTOR', 'STOCK', 'DATE']
    feature_gbs = []
    for i in range(1, len(feature_to_gbs) + 1):
        feature_gbs += list(combinations(x_train[feature_to_gbs], i))
    meanx_train = weekly_means(x_train)
    target_feature = 'average R Week'
    for index in range(4):
        for i in range(len(feature_gbs)):
            feat = f'{target_feature} {index}'
            tmp_name = '_'.join(feature_gbs[i])
            meanx_train[f'{target_feature}_{index}_{tmp_name}_mean'] = meanx_train.groupby(list(feature_gbs[i]))[
                feat].transform('mean')
    return meanx_train

In [13]:
weekly_means_train = data_permute(x_train)
weekly_means_test = data_permute(x_test)

In [94]:
def build_and_compile_model():
  model = keras.Sequential([
      layers.Dense(128, input_shape=(73,), activation='relu'),
      layers.Dense(128, activation='relu'),
      layers.Dense(1)
  ])

  model.compile(loss='binary_crossentropy',
                metrics=['accuracy'],
                optimizer=tf.keras.optimizers.Nadam(0.001))
  return model

dnn_model = build_and_compile_model()
dnn_model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)            (None, 128)               9472      
                                                                 
 dense_16 (Dense)            (None, 128)               16512     
                                                                 
 dense_17 (Dense)            (None, 1)                 129       
                                                                 
Total params: 26,113
Trainable params: 26,113
Non-trainable params: 0
_________________________________________________________________


In [95]:
history = dnn_model.fit(
    weekly_means_train.iloc[:,2:], y_train['RET'],
    epochs=10,
    batch_size=256,
    validation_split=0.2,
    verbose=1)

Epoch 1/10
1309/1309 [==============================] - 5s 3ms/step - loss: 7.6958 - accuracy: 0.5011 - val_loss: 7.6962 - val_accuracy: 0.5011
Epoch 2/10
1309/1309 [==============================] - 3s 2ms/step - loss: 7.6958 - accuracy: 0.5011 - val_loss: 7.6962 - val_accuracy: 0.5011
Epoch 3/10
1309/1309 [==============================] - 3s 2ms/step - loss: 7.6958 - accuracy: 0.5011 - val_loss: 7.6962 - val_accuracy: 0.5011
Epoch 4/10
1309/1309 [==============================] - 3s 2ms/step - loss: 7.6958 - accuracy: 0.5011 - val_loss: 7.6962 - val_accuracy: 0.5011
Epoch 5/10
1309/1309 [==============================] - 3s 2ms/step - loss: 7.6958 - accuracy: 0.5011 - val_loss: 7.6962 - val_accuracy: 0.5011
Epoch 6/10
1309/1309 [==============================] - 3s 2ms/step - loss: 7.6958 - accuracy: 0.5011 - val_loss: 7.6962 - val_accuracy: 0.5011
Epoch 7/10
1309/1309 [==============================] - 3s 2ms/step - loss: 7.6958 - accuracy: 0.5011 - val_loss: 7.6962 - val_accuracy:

In [98]:
target = 'RET'
y_pred = dnn_model.predict(weekly_means_test.iloc[:, 2:])

sub = weekly_means_test.copy()
sub['pred'] = y_pred
y_pred = sub.groupby('DATE')['pred'].transform(
    lambda x: x > x.median()).values

submission = pd.Series(y_pred)
submission.index = x_test['ID']
submission.name = target

submission.to_csv('dnn2.csv', index=True, header=True)

In [109]:
y_rf = pd.read_csv('y_pred_rforestbest.csv')
y_xg = pd.read_csv('xg_boost20.csv')
y_nn = pd.read_csv('dnn2.csv')
y_means = []
for i in range(y_rf.shape[0]):
    s = np.round((y_rf['RET'][i]+y_nn['RET'][i]+y_xg['RET'][i])/3,0)
    y_means.append(s)
y_means = pd.Series(y_means)
y_means.index = x_test['ID']
y_means.name = 'RET'
y_means.to_csv('ensemble.csv', index=True, header=True)

In [137]:
def build_and_compile_model3():
  model = keras.Sequential([

      layers.LSTM(128, input_shape=(73,1),activation='selu'),
      layers.AlphaDropout(0.05),
      layers.Dense(64, activation='selu'),
      layers.Dense(1)
  ])

  model.compile(loss='binary_crossentropy',
                metrics=['accuracy'],
                optimizer=tf.keras.optimizers.Nadam(0.1))
  return model

lnn_model = build_and_compile_model3()
lnn_model.summary()

Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_14 (LSTM)              (None, 128)               66560     
                                                                 
 alpha_dropout (AlphaDropout  (None, 128)              0         
 )                                                               
                                                                 
 dense_46 (Dense)            (None, 64)                8256      
                                                                 
 dense_47 (Dense)            (None, 1)                 65        
                                                                 
Total params: 74,881
Trainable params: 74,881
Non-trainable params: 0
_________________________________________________________________


In [138]:
history = lnn_model.fit(
    tf.expand_dims(weekly_means_train.iloc[:,2:],axis=-1), y_train['RET'],
    epochs=10,
    batch_size=1024,
    validation_split=0.2,
    verbose=1)

Epoch 1/10
328/328 [==============================] - 167s 501ms/step - loss: 7.6858 - accuracy: 0.4986 - val_loss: 7.6100 - val_accuracy: 0.5022
Epoch 2/10
328/328 [==============================] - 165s 503ms/step - loss: 7.6994 - accuracy: 0.4974 - val_loss: 7.6100 - val_accuracy: 0.5022
Epoch 3/10
328/328 [==============================] - 173s 526ms/step - loss: 7.6874 - accuracy: 0.4982 - val_loss: 7.6100 - val_accuracy: 0.5022
Epoch 4/10
328/328 [==============================] - 178s 542ms/step - loss: 7.7017 - accuracy: 0.4972 - val_loss: 7.6100 - val_accuracy: 0.5022
Epoch 5/10
328/328 [==============================] - 161s 490ms/step - loss: 7.6655 - accuracy: 0.4996 - val_loss: 7.6100 - val_accuracy: 0.5022
Epoch 6/10
328/328 [==============================] - 164s 500ms/step - loss: 7.6870 - accuracy: 0.4982 - val_loss: 7.6100 - val_accuracy: 0.5022
Epoch 7/10
328/328 [==============================] - 154s 469ms/step - loss: 7.6833 - accuracy: 0.4985 - val_loss: 7.6100 -

In [146]:
y_lstm = np.sign(lnn_model.predict(tf.expand_dims(weekly_means_test.iloc[:,2:],axis=-1)))

In [150]:
type(y_lstm)

numpy.ndarray

In [151]:
submission = pd.Series(y_lstm[:,0])
submission.index = x_test['ID']
submission.name = target

submission.to_csv('lstm.csv', index=True, header=True)